# Evaluation scrip to run on NXP board

## Save images to png

inspirated by https://stackoverflow.com/questions/57338091/how-to-save-tensor-as-a-image

In [21]:
import tensorflow as tf
import tensorflow_datasets as tfds
import os
from PIL import Image
import pathlib

def preprocess_flowers_test_and_val(image, label):
    image = tf.image.resize(image, [256, 256],
                          method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    return image, label


(ds_tran, ds_test), ds_info = tfds.load(
        'oxford_flowers102',
        split=['train','test'],
        as_supervised=True,
        with_info=True,
    )
ds_test = ds_test.map(preprocess_flowers_test_and_val)

i = 0
for image, label in ds_test.as_numpy_iterator():
    i+=1
    img = Image.fromarray(image)
    image_directory = pathlib.Path("../datasets/flowers_test/" + str(label))
    image_directory.mkdir(parents=True, exist_ok=True)
    image_path = image_directory/(str(i) + '.png')
    img.save(str(image_path))

In [22]:
(ds_tran, ds_test), ds_info = tfds.load(
        'beans',
        split=['train','test'],
        as_supervised=True,
        with_info=True,
    )

i = 0
for image, label in ds_test.as_numpy_iterator():
    i+=1
    img = Image.fromarray(image)
    image_directory = pathlib.Path("../datasets/beans_test/" + str(label))
    image_directory.mkdir(parents=True, exist_ok=True)
    image_path = image_directory/(str(i) + '.png')
    img.save(str(image_path))

# Evaluation script to run on mobile device

In [1]:
import time

import numpy as np
from PIL import Image
import json
import copy
from json import JSONEncoder
import os
import os
import zipfile
import tempfile
import sys

# todo uncomment this on device and comment line bellow
import tflite_runtime.interpreter as tflite


# import tensorflow.lite as tflite


class EvaluatedModel:
    y_pred = []
    y_true = []
    model_name = ''
    avg_time = 0.0
    gzip_size = 0

    def __init__(self, dict1=None, y_pred=[], y_true=[], model_name='', avg_time=0.0, gzip_size=0):
        if (dict1 == None):
            self.y_pred = copy.deepcopy(y_pred)
            self.y_true = copy.deepcopy(y_true)
            self.model_name = model_name
            self.avg_time = avg_time
            self.gzip_size = gzip_size
        else:
            self.__dict__.update(dict1)


# subclass JSONEncoder
class EvaluatedModelEncoder(JSONEncoder):
    def default(self, o):
        return o.__dict__


def main(argv):
    if len(argv) != 2:
        print('args: beans/flowers MobileNetV2/EfficentNetB0')
        sys.exit(2)
    # parse arguments
    dataset = argv[0]
    model = argv[1]

    def get_gzipped_model_size(file):
        # Returns size of gzipped model, in bytes.

        _, zipped_file = tempfile.mkstemp('.zip')
        with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
            f.write(file)

        return os.path.getsize(zipped_file)

    def evaluate_model(model_path, dataset_path):
        """ returns EvaluatedModel for given model and dataset """

        # model cannot use
        if "dynamic_rage_quantization" in model_path:
            print("model cannot use  NNAPI\n\n")
            return EvaluatedModel(dict1=None,
                                  y_pred=[],
                                  y_true=[],
                                  model_name=model_path.split('/')[-1],
                                  avg_time=0,
                                  gzip_size=0)

        interpreter = tflite.Interpreter(
            model_path=model_path, num_threads=None)
        interpreter.allocate_tensors()

        input_details = interpreter.get_input_details()
        output_details = interpreter.get_output_details()

        # check the type of the input tensor
        floating_model = input_details[0]['dtype'] == np.float32 or input_details[0]['dtype'] == np.float16
        if not floating_model:
            print('model has int i/o')

        input_index = input_details[0]["index"]

        y_true = []
        y_pred = []
        avg_time = 0.0

        for subdir, dirs, files in os.walk(dataset_path):
            for file in files:
                if file == '.DS_Store':
                    continue

                y_true.append(int(subdir.split('/')[-1]))

                img = Image.open(os.path.join(subdir, file))

                # add batch dimension
                test_image = np.expand_dims(img, axis=0)

                if floating_model:
                    test_image = (np.float32(test_image) / 255.0)

                interpreter.set_tensor(input_details[0]['index'], test_image)

                # ignore the 1st invoke
                interpreter.invoke()

                startTime = time.time()
                interpreter.invoke()
                delta = (time.time() - startTime) * 1000
                avg_time += delta

                output = interpreter.get_tensor(output_details[0]['index'])

                # remove batch dimension and return predicted label
                y_pred.append(np.argmax(output[0]).item())

        gzip_size = get_gzipped_model_size(model_path)

        evaluated_model = EvaluatedModel(dict1=None,
                                         y_pred=y_pred,
                                         y_true=y_true,
                                         model_name=model_path.split('/')[-1],
                                         avg_time=avg_time / len(y_pred),
                                         gzip_size=gzip_size)
        print('avg time: ' + str(evaluated_model.avg_time) + 'ms, gzip: ' + str(evaluated_model.gzip_size) + 'Bytes\n')

        return evaluated_model

    dataset_path = 'datasets/' + dataset + '_test/'
    print(dataset_path)
    evaluated_models = []

    for subdir, dirs, files in os.walk(dataset + '_models_optimized/'):
        model_no = 0
        for file in files:

            if file.split('.')[-1] == 'tflite' and model in file:
                model_no += 1
                # check if model is in tflite
                print('evaluating: ' + file)
                evaluated_models.append(evaluate_model(os.path.join(subdir, file), dataset_path))
                if model_no % 10 == 0:
                    encoded_json = EvaluatedModelEncoder().encode(evaluated_models)
                    with open('evaluated_' + dataset + '_models_on_' + model + str(model_no) + '.json', 'w') as f:
                        f.write(encoded_json)

    encoded_json = EvaluatedModelEncoder().encode(evaluated_models)
    with open('evaluated_' + dataset + '_models_on_' + model + '.json', 'w') as f:
        f.write(encoded_json)


if __name__ == "__main__":
    main(sys.argv[1:])


evaluating: MobileNetV2_flowers_model_PolynomialDecay50_float16_quantization.tflite


ZeroDivisionError: float division by zero